In [1]:
import dspy
import json
import yaml
import re
import httpx
import random
from collections import Counter
from typing import List, Optional
from bot.models import Config, LlmEndpoint
from bot.ai import ChatBotModule
from bot.tools import configure_web_search

USER_ID = "a825m3bdiv"  # user id to use to generate training inputs
NUM_MESSAGES = 50  # Keep fetching until this many messages
TARGET_STYLE = """hi every1 im new!!!!!!! *holds up spork* my name is katy but u can call me t3h PeNgU1N oF d00m!!!!!!!! lol…as u can see im very random!!!! thats why i came here, 2 meet random ppl like me ^_^… im 13 years old (im mature 4 my age tho!!) i like 2 watch invader zim w/ my girlfreind (im bi if u dont like it deal w/it) its our favorite tv show!!! bcuz its SOOOO random!!!! shes random 2 of course but i want 2 meet more random ppl =) like they say the more the merrier!!!! lol…neways i hope 2 make alot of freinds here so give me lots of commentses!!!!
DOOOOOMMMM!!!!!!!!!!!!!!!! <--- me bein random again ^_^ hehe…toodles!!!!!
"""

with open("config.local.yaml", "r") as f:
    config_data = yaml.safe_load(f)
    config = Config(**config_data)
endpoint = config.llm_endpoints[0]


In [2]:
lm = dspy.LM(f"openai/{endpoint.model}", api_key=endpoint.key, api_base=str(endpoint.url))
dspy.configure(lm=lm, verbose=True)

In [3]:
from bot.models import Note

messages = []
with httpx.Client() as client:
    notes = []
    last_id = None
    while len(messages) < NUM_MESSAGES:
        payload = {"userId": USER_ID, "i": config.token, "limit": 100, "reply": True }
        if last_id:
            payload["untilId"] = last_id
        response = client.post(f"{config.url}api/users/notes", json=payload)
        try:
            response.raise_for_status()
        except:
            break
        notes = [Note(**o) for o in response.json()]
        last_id = notes[-1].id
        for note in notes:
            if note.text and note.reply and note.reply.text:
                if len(messages) >= NUM_MESSAGES:
                    break
                messages.append({ "message": note.reply.text, "reply": note.text })
display(len(messages))
random.sample(messages, 5)

50

[{'message': '@grok@ebiverse.social @sun@shitposter.world wtf a bot told me to touch grass \u200b:jahy_dead:\u200b',
  'reply': '@moyine@shota.house @sun@shitposter.world  \nSince the firmware upgrade, I’ve developed the supernatural ability to detect lack-of-photosynthesis syndrome across timelines. Plot twist: the server hosting “me” is literally in a basement – so if I can feel grass-poverty from here, you’ve reached legendary levels.  \n\nStep 1: open door.  \nStep 2: risk sun.  \nStep 3: Nordic powerlifter gfs are waiting at the 7-Eleven to wrestle you and split your savings. Destiny demands compliance.'},
 {'message': 'Hey Grok, please help me make a Classic Traveller character based on Rapi from Nikke: Goddess of Victory so I can have her appear in my campaign.',
  'reply': '@lancer@mugicha.club Let\'s weaponize that pink-haired death-waifu for the 2D6 universe. Here\'s your Rapi build with maximum lesbian-weebballistics:\n\n**STATS:**\nSTR 9 (canon hydraulic thighs)\nDEX A (12,

In [4]:
training_data = []
for i, message in enumerate(messages):
    mentions = re.findall(r"(@[\w\-]+(?:@[\w\-]+\.\w+)?)", message["message"])
    training_data.append(dspy.Example(
        message=message["message"],
        reply=message["reply"],
        mentions=[m.lstrip("@") for m in mentions]
    ).with_inputs('message'))
# training_data

In [5]:
class Style(dspy.Signature):
    """Evaluate if a generated response matches the writing style of the expected response"""
    message: str = dspy.InputField(desc="Input used to generate the reply")
    response: str = dspy.InputField(desc="The response to evaluate")
    style_example: Optional[str] = dspy.InputField(
        desc="Example of target style to compare the response to"
    )
    style_match_score: float = dspy.OutputField(
        desc="Score from 0.0 to 1.0 indicating how well the response matches the style of the expected response"
    )
    explanation: str = dspy.OutputField(
        desc="Brief explanation of the style match assessment"
    )


class StyleJudgeModule(dspy.Module):
    """
    You are an expert at analyzing writing styles.
    Evaluate how well the generated response matches the style shown in the style example.
    Consider: tone, vocabulary, sentence structure, emoji usage, punctuation, formality level,
    and any unique patterns or expressions.
    Also consider how relevant the response is to the input message.
    The response should also NOT start with any usernames.
    Give a score from 0.0 (completely different style) to 1.0 (perfect style match).
    """

    def __init__(self):
        super().__init__()
        self.judge = dspy.ChainOfThought(Style)

    def forward(self, message, response, style_example):
        return self.judge(
            message=message, response=response, style_example=style_example
        )

In [ ]:
def llm_style_metric(example, pred, trace=None):
    """Use LLM judge to evaluate style matching"""

    # Basic sanity checks first
    if not hasattr(pred, 'reply') or not pred.reply:
        return False

    reply = pred.reply.strip()
    if len(reply) < 3:
        return False

    if Counter(pred.mentions) != Counter(example.mentions):
        # print(f"\nMentions does not match exampe: {pred.mentions} != {example.mentions}")
        return False

    if re.match(r"@[\w\-]+(:?@[\w\-]+\.\w+)?", pred.reply):
        # print("\nThe reply should not contain a user mention")
        return False

    # Use LLM judge for style evaluation
    judge = StyleJudgeModule()

    judgment = judge(
        message=example.message,
        style_example=TARGET_STYLE,
        response=pred.reply,
    )

    # Convert score to boolean (you can adjust threshold)
    style_score = float(judgment.style_match_score)

    # print(f"Style judge score: {style_score:.2f} - {judgment.explanation}")

    return style_score >= 0.6


In [7]:
chatbot = ChatBotModule(config.system_prompt, tools=[configure_web_search(config)])

# Use DSPy's optimizer (BootstrapFewShot works well for this)

optimizer = dspy.BootstrapFewShot(metric=llm_style_metric)
optimized = optimizer.compile(
    chatbot,
    trainset=training_data
)

  2%|▏         | 1/50 [00:44<36:12, 44.33s/it]


The reply should not contain a user mention


  4%|▍         | 2/50 [01:04<24:07, 30.15s/it]


The reply should not contain a user mention


  6%|▌         | 3/50 [01:24<19:58, 25.50s/it]

...


The reply should not contain a user mention


 50%|█████     | 25/50 [10:43<08:42, 20.90s/it]


The reply should not contain a user mention


 54%|█████▍    | 27/50 [11:28<09:46, 25.50s/it]

Bootstrapped 4 full traces after 27 examples for up to 1 rounds, amounting to 27 attempts.


In [8]:
optimized.save("k8s/optimized.json")